# Surface diffusion energy barriers using the Nudged Elastic Band (NEB) method
https://wiki.fysik.dtu.dk/ase/tutorials/neb/diffusion.html
First, set up the initial and final states:

In [1]:
from ase.build import fcc100, add_adsorbate
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton

2x2-Al(001) surface with 3 layers and an Au atom adsorbed in a hollow site:

In [2]:
slab = fcc100('Al', size=(2, 2, 3))
add_adsorbate(slab, 'Au', 1.7, 'hollow')
slab.center(axis=2, vacuum=4.0)

Make sure the structure is correct: `view(slab)`

Fix second and third layers:

In [3]:
mask = [atom.tag > 1 for atom in slab]
print(mask)
slab.set_constraint(FixAtoms(mask=mask))

[True, True, True, True, True, True, True, True, False, False, False, False, False]


In [4]:
slab.set_calculator(EMT())

Initial state:

In [5]:
qn = QuasiNewton(slab, trajectory='initial.traj')
qn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 18:13:41        3.323870*       0.2462
BFGSLineSearch:    1[  1] 18:13:41        3.314754*       0.0378


True

Final state:

In [6]:
slab[-1].x += slab.get_cell()[0, 0] / 2
qn = QuasiNewton(slab, trajectory='final.traj')
qn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 18:13:42        3.320051*       0.1208
BFGSLineSearch:    1[  1] 18:13:42        3.316117*       0.0474


True

Now, do the NEB calculation:

In [7]:
from ase.io import read
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.neb import NEB
from ase.optimize import BFGS

In [8]:
initial = read('initial.traj')
final = read('final.traj')

In [9]:
constraint = FixAtoms(mask=[atom.tag > 1 for atom in initial])

In [10]:
images = [initial]
for i in range(3):
    image = initial.copy()
    image.set_calculator(EMT())
    image.set_constraint(constraint)
    images.append(image)

In [11]:
images.append(final)

In [ ]:
neb = NEB(images)
neb.interpolate()
qn = BFGS(neb, trajectory='neb.traj')
qn.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 18:13:46        4.219952        3.5208
BFGS:    1 18:13:49        3.983963        2.4270
BFGS:    2 18:13:52        3.830638        1.5293


You can also analyze within a python script, which can be useful particularly if you are analyzing the output of many NEB jobs, with the `ase.neb.NEBTools` class. Some examples of its use are below; the final example was used to make the figure you see above.

In [ ]:
import matplotlib.pyplot as plt
from ase.neb import NEBTools
from ase.io import read

In [ ]:
images = read('neb.traj@-5:')

In [ ]:
nebtools = NEBTools(images)

Get the calculated barrier and the energy change of the reaction.

In [ ]:
Ef, dE = nebtools.get_barrier()

Get the barrier without any interpolation between highest images.

In [ ]:
Ef, dE = nebtools.get_barrier(fit=False)

Get the actual maximum force at this point in the simulation.

In [ ]:
max_force = nebtools.get_fmax()

Create a figure like that coming from ASE-GUI.

In [ ]:
fig = nebtools.plot_band()
fig.savefig('diffusion-barrier.png')

Create a figure with custom parameters.

In [ ]:
fig = plt.figure(figsize=(5.5, 4.0))
ax = fig.add_axes((0.15, 0.15, 0.8, 0.75))
nebtools.plot_band(ax)
fig.savefig('diffusion-barrier.png')